In [ ]:
!pip uninstall tensorflow -y 
!pip install tensorflow==1.15

In [ ]:
!wget https://raw.githubusercontent.com/google-research/bert/master/modeling.py 
!wget https://raw.githubusercontent.com/google-research/bert/master/optimization.py
!wget https://raw.githubusercontent.com/google-research/bert/master/run_classifier.py
!wget https://raw.githubusercontent.com/google-research/bert/master/tokenization.py

In [2]:
import numpy as np
import re
import os
import pandas as pd 
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import tensorflow as tf

from datetime import datetime

# BERT
import optimization
import run_classifier
import tokenization
import tensorflow_hub as hub

In [3]:
tf.__version__

'1.15.0'

# Preparing Data

## Drive operations

In [4]:
# Mount Google Drive
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT, force_remount=True)           # we mount the google drive at /content/drive

/content/drive
Mounted at /content/drive


In [5]:
%cd 'drive/My Drive/Workspaces/bert_document_classification'

/content/drive/My Drive/Workspaces/bert_document_classification


## Getting dataframe

### Creating Dataframe an storing

In [16]:
labels = []
filenames = []
texts = []
titles = []
data_dict = {}

for category in os.listdir('data'):
  category_path  = os.path.join('data',category)

  for filename in os.listdir(category_path):
    file_path = os.path.join(category_path,filename)
       
    with open(file_path,'r', encoding='utf8', errors='ignore') as text_file:
      text = ''
      firstline = True
      for line in text_file:
        if firstline:
            titles.append(line.replace('\n',''))
            firstline = False
        else:
            text = text + ' ' + line.replace('\n','')

      texts.append(text)
      labels.append(category)
      filenames.append(filename)

data_dict['text'] = texts
data_dict['title'] = titles
data_dict['label'] = labels
data_dict['filename'] = filenames

In [ ]:
data_df = pd.DataFrame(data_dict)
data_df.head()


In [18]:
# saving the data set
import pickle
pickle_out = open('data/data_df.pickle','wb')
pickle.dump(data_df,pickle_out)
pickle_out.close()

### Retrieving dataframe from stored file

In [ ]:
pickle_in = open('data/data_df.pickle','rb')
data_df = pickle.load(pickle_in)
pickle_in.close()

In [ ]:
df = data_df[['text','label']]
df.head()

# EDA

In [21]:
df.isnull().any()

text     False
label    False
dtype: bool

No null values

In [23]:
df.label.value_counts()

sport            511
business         510
politics         417
tech             401
entertainment    386
Name: label, dtype: int64